In [1]:
import numpy as np , pandas as pd , matplotlib.pyplot as plt

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv(r"C:\Users\ASUS\Downloads\uber_rides_data.xlsx - sample_train.csv")
data.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


In [4]:
x = data['fare_amount'].dropna()
x.mean()

11.359955250000002

In [5]:
data.head(2)

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1


## Which of the following is the correct syntax to convert 'pickup_datetime' to datetime datatype?

In [6]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])

## Calculate distance between each pickup and dropoff points using Haversine formula. What is the median haversine distance between pickup and dropoff location according to the given dataset? 

In [7]:
# Function to calculate Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

# Calculate Haversine distances for each record
data['haversine_distance'] = data.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], 
                                                              row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

# Calculate the median Haversine distance
median_distance = data['haversine_distance'].median()
print("Median Haversine Distance:", median_distance, "kilometers")

Median Haversine Distance: 2.1209923961833708 kilometers


In [8]:
data.head(2)

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_distance
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590


## What is the maximum haversine distance between pickup and dropoff location according to the given dataset?

In [9]:
# Max distance:
data['haversine_distance'].max()

16409.239135313168

## How many rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset?

In [10]:
count_zero_distance_rides = len(data[data['haversine_distance'] == 0.0])

print(f'{count_zero_distance_rides} rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset')

5632 rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset


## What is the mean 'fare_amount' for rides with 0 haversine distance?

In [11]:
mean_fare_amount_zero_distance = data[data['haversine_distance'] == 0.0]['fare_amount'].mean()

print("Mean 'fare_amount' for rides with 0.0 Haversine distance:", mean_fare_amount_zero_distance)

Mean 'fare_amount' for rides with 0.0 Haversine distance: 11.585317826704546


+ Rides with a Haversine distance of 0.0 are unusual and could indicate problems with the data
+ Zero distance suggests that the pickup and dropoff locations are the same, which could be due to incorrect data entry, missing GPS coordinates etc.

## What is the maximum 'fare_amount' for a ride?

In [12]:
max_amt = data['fare_amount'].max()
print(f'Max fare amount : {max_amt}')

Max fare amount : 499.0


## What is the haversine distance between pickup and dropoff location for the costliest ride?

In [13]:
# Sort the dataset by 'fare_amount' in descending order and retrieve the top row
costliest_ride = data.sort_values(by='fare_amount', ascending=False).iloc[0]

pickup_coords = (costliest_ride['pickup_latitude'], costliest_ride['pickup_longitude'])
dropoff_coords = (costliest_ride['dropoff_latitude'], costliest_ride['dropoff_longitude'])


haversine_distance = haversine(*pickup_coords, *dropoff_coords)

print("Haversine Distance for Costliest Ride:", haversine_distance, "kilometers")


Haversine Distance for Costliest Ride: 0.0007899213191009994 kilometers


## How many rides were recorded in the year 2014?


In [14]:
rides_2014 = data[data['pickup_datetime'].dt.year==2014]

print("Number of rides in the year 2014:", len(rides_2014))

Number of rides in the year 2014: 29968


## How many rides were recorded in the first quarter of 2014?

In [15]:
rides_1st_q = data[(data['pickup_datetime'] >= '2014-1-1') & (data['pickup_datetime'] <= '2014-04-1')]

print("Number of rides in the 1st quarter of year 2014:", len(rides_1st_q))

Number of rides in the 1st quarter of year 2014: 7687


## On which day of the week in September 2010, maximum rides were recorded ?

In [16]:
data['ride_week_day'] = data['pickup_datetime'].dt.day_name()

In [17]:
data.head(2)

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_distance,ride_week_day
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,Thursday
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,Friday


In [18]:
rides_september_2010 = data[(data['pickup_datetime'] >= '2010-09-01') & (data['pickup_datetime'] <= '2010-10-1')]


day_of_week_counts = rides_september_2010['ride_week_day'].value_counts()


max_day = day_of_week_counts.idxmax()
max_count = day_of_week_counts[max_day]

print(f"On {max_day} in September 2010, the maximum number of rides were recorded: {max_count} rides")

On Thursday in September 2010, the maximum number of rides were recorded: 457 rides


# Apply a Machine Learning Algorithm to predict the fare amount given following input features:

In [19]:
data = data.dropna()
data.isna().sum()

ride_id               0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
passenger_count       0
haversine_distance    0
ride_week_day         0
dtype: int64

In [23]:
data.head(5)

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_distance,ride_week_day
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,Thursday
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,Friday
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,Monday
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,Friday
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,Thursday


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,RobustScaler,StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [43]:
# Split the data into features (X) and the target variable (y)
cont_col = ['passenger_count', 'haversine_distance']
nom_col = [ 'ride_week_day']

x = data[cont_col+nom_col]
y = data['fare_amount']

# Perform a 70-30 split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Preprocessing the data

preprocessor = ColumnTransformer(transformers = [
            ('cont_pipeline' , Pipeline([
               ( 'cont_scale', StandardScaler())
            ]), cont_col),
            
            ('nom_pipeline', Pipeline([
                ('nom_encode',OneHotEncoder())
            ]),nom_col)
],remainder = 'passthrough')

x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


In [44]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regression': RandomForestRegressor(),
    'KNN Regressor': KNeighborsRegressor(),
    'Decission Tree Regressor':DecisionTreeRegressor()
}

adjusted_r2_values = {}

for model_name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = x_test.shape[1]
    
    adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
    adjusted_r2_values[model_name] = adjusted_r2

# Find the algorithm with the least adjusted R-squared value
least_adjusted_r2_algorithm = min(adjusted_r2_values, key=adjusted_r2_values.get)

print("Adjusted R-squared values:")
for model_name, adjusted_r2 in adjusted_r2_values.items():
    print(f"{model_name}: {adjusted_r2}")

print("Algorithm with the least adjusted R-squared value:", least_adjusted_r2_algorithm)

Adjusted R-squared values:
Linear Regression: 0.0005965473953365441
Random Forest Regression: 0.6243405517039873
KNN Regressor: 0.6245388696014144
Decission Tree Regressor: 0.4769157204429203
Algorithm with the least adjusted R-squared value: Linear Regression
